# Day 3: Land Use

Download the datasets:

```
overturemaps download -f geoparquet --type=land -o $HOME/data/overture/land.parquet

overturemaps download -f geoparquet --type=land_use -o $HOME/data/overture/land_use.parquet
```

```
overturemaps download -f geoparquet --type=land_use -o land_use.parquet
```

In [15]:
import geopandas as gpd
from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_categorical_cmap
import pandas as pd
import numpy as np
import wkls

import sedona.db

In [16]:
sd = sedona.db.connect()

In [17]:
land_df = sd.read_parquet(
    "/Users/matthewpowers/data/overture/land.parquet"
)
land_df.to_view('land')

land_use_df = sd.read_parquet(
    "/Users/matthewpowers/data/overture/land_use.parquet"
)
land_use_df.to_view('land_use')

In [18]:
area = wkls.us.ny.cityofnewyork.wkt()

In [19]:
land_use_filtered = sd.sql(f"""
SELECT *
FROM land_use
WHERE ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{area}'), 4326))
""")
land_use_filtered.to_view('land_use_area', True)

In [20]:
classified = sd.sql("""
SELECT 
    id,
    class,
    subtype,
    geometry,
    CASE 
        WHEN class = 'residential' THEN 'Residential'
        WHEN class = 'commercial' THEN 'Commercial'
        WHEN class = 'industrial' THEN 'Industrial'
        WHEN class = 'recreation' THEN 'Recreation'
        WHEN class = 'park' THEN 'Parks'
        WHEN class = 'education' THEN 'Education'
        WHEN class = 'institutional' THEN 'Institutional'
        WHEN class = 'transportation' THEN 'Transportation'
        WHEN class = 'agriculture' THEN 'Agriculture'
        WHEN class = 'forest' THEN 'Forest'
        WHEN class = 'water' THEN 'Water'
        ELSE 'Other'
    END as land_use_category
FROM land_use_area
WHERE geometry IS NOT NULL
""")

In [21]:
gdf = classified.to_pandas()
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [22]:
color_map = {
    'Residential': [255, 247, 188, 200],      # Light yellow
    'Commercial': [255, 99, 71, 200],         # Red
    'Industrial': [147, 112, 219, 200],       # Purple
    'Recreation': [144, 238, 144, 200],       # Light green
    'Parks': [34, 139, 34, 200],              # Forest green
    'Education': [135, 206, 235, 200],        # Sky blue
    'Institutional': [70, 130, 180, 200],     # Steel blue
    'Transportation': [128, 128, 128, 200],   # Gray
    'Agriculture': [255, 215, 0, 200],        # Gold
    'Forest': [0, 100, 0, 200],               # Dark green
    'Water': [65, 105, 225, 200],             # Royal blue
    'Other': [211, 211, 211, 200]             # Light gray
}

In [23]:
gdf['color'] = gdf['land_use_category'].map(color_map)

In [24]:
gdf_polygons = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])].copy()

colors = np.array(gdf_polygons['color'].tolist(), dtype='uint8')

layer = PolygonLayer.from_geopandas(
    gdf_polygons,
    get_fill_color=colors,
    get_line_color=[255, 255, 255, 100],
    line_width_min_pixels=0.5,
    pickable=True
)

In [26]:
map = Map(
    layers=[layer],
    # view_state={
    #     'latitude': 33.45,
    #     'longitude': -112.1,
    #     'zoom': 10,
    #     'pitch': 0
    # }
)

map

Map(custom_attribution='', layers=(PolygonLayer(get_fill_color=arro3.core.ChunkedArray<FixedSizeList(Field { n…